In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [4]:
import numpy as np 
from numpy import linspace
import pandas as pd


from datetime import datetime,timedelta
from pathlib import Path



In [12]:
from pre_process_func import read_queue
from fileUtils import read_file_properties
from labeling_utils import load_labels

from visUtils import file2TableDict,createTimeIndex#,get_cycle,reverseTableDict
from visUtils import rawFile2Csv

from fileUtils import list_files,standardPathStyle


In [6]:
flder="/home/enis/projects/nna/data/"

# files_list_path=flder+"mp3files.txt"
# fileswlen_path = flder+"mp3files_wlen.txt"

# files_list_path=flder+"flacfiles.txt"
# fileswlen_path = flder+"flacfiles_wlen.txt"

# coordinates_fpath= flder + "NNA_recording_devices.csv"

In [7]:
file_properties_df=pd.read_pickle("../../data/stinchcomb_dataV1.pkl")

all_areas=sorted(pd.unique(file_properties_df.site_id.values))

In [24]:
# PARAMS
# FREQS to reduce results 
dataFreq="10S"
# freq="T"
freq="10S"

# clipping threshold
threshold=0.01  # if x <= then 0

# possible places to pick
# sorted(pd.unique(file_properties_df.site_id.values))
# areas to be visualized

# delete older than 2016
fromtime=datetime(2016, 1, 1, 0)

result_path="/scratch/enis/data/nna/real/"

selected_areas=all_areas[:]


tag_set=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

model_tag_names=["CABLE","RUNNINGWATER","INSECT", "RAIN", "WATERBIRD", "WIND", "SONGBIRD", "AIRCRAFT"]

model_tag_names=["_"+i for i in model_tag_names]


In [25]:
#important to keep them in order
file_properties_df.sort_values(by=['timestamp'],inplace=True)

# delete olde r than 2016
fromtime=datetime(2016, 1, 1, 0)
file_properties_df=file_properties_df[file_properties_df.timestamp>=fromtime]


In [26]:
len(file_properties_df)

359

In [ ]:
%%time

################# Saving 10S directly 
freq="10S"

globalcolumns=model_tag_names#selected_areas+weather_cols

globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)


saveCsvPath=Path("/home/enis/projects/nna/results/ExternalProject/Todd-Megan/Freq-10S/")
saveCsvPath.mkdir(parents=True,exist_ok=True)

csvFilesWritten,no_result_paths=rawFile2Csv(saveCsvPath,selected_areas,model_tag_names,
                                            globalindex,globalcolumns,file_properties_df,
                                            freq,dataFreq,result_path=result_path,prob2binaryFlag=False)


16-FishCreek2, do not have any files
20-Umiat, do not have any files


In [235]:
%%time

################# Saving as grouped by frequency  
freq="2H"


globalcolumns=model_tag_names#selected_areas+weather_cols

globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)

saveCsvPath=Path("/home/enis/projects/nna/results/ExternalProject/Todd-Megan/Freq-2H/")
saveCsvPath.mkdir(parents=True,exist_ok=True)

df_dict,no_result_paths=file2TableDict(selected_areas,model_tag_names,globalindex,
                                       globalcolumns,file_properties_df,freq,dataFreq,
                                       result_path=result_path,prob2binaryFlag=False)


for key,val in df_dict.items():
    print(key)
    df_count,df_sums=val
    res=(df_sums/df_count).fillna(0.0)
    for col in res.columns:
        csvFilename="_".join([key,col[1:]+".csv"])
        res.to_csv(saveCsvPath/csvFilename,columns=[col],float_format='%.2f',index_label="TimeStamp",header=[col[1:]])


CPU times: user 2.1 s, sys: 180 ms, total: 2.28 s
Wall time: 2.28 s


1
